# 📊 Notebook de Análisis y Persistencia de `sales_data_sample`

Este notebook en **Databricks + PySpark** carga, procesa y analiza el archivo 
`sales_data_sample.csv`.  

Incluye:
- ✅ Lectura 
- ✅ Creación de vistas temporales y persistentes  
- ✅ Consultas con joins y agregaciones  
- ✅ Ejemplos de funciones de ventana explicadas en detalle  

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Ruta por defecto (ajusta si tu CSV está en otra ubicación)
file_path = "/Volumes/workspace/dbtest/dataclase3"

# Lectura del CSV con header=True y separador ','
df_sales = spark.read.option("header", "true") \
.option("sep", ",") \
.option("inferSchema", "true") \
.csv(file_path)

# Mostrar esquema y primeras filas
print("Esquema detectado:")
df_sales.printSchema()

# En Databricks es práctico usar display()
display(df_sales.limit(10))

## Creación de Vistas Temporales y Globales
En Spark/Databricks podemos exponer DataFrames como vistas SQL:

- **Vista Temporal**: solo disponible en la sesión actual.  
- **Vista Global Temporal**: accesible desde todas las sesiones dentro de la aplicación Spark.  

In [0]:

# Vista temporal
df_sales.createOrReplaceTempView("vw_sales_temp")

# Vista global
df_sales.createOrReplaceGlobalTempView("vw_sales_global")

print("✅ Vistas creadas: vw_sales_temp (temporal) y global_temp.vw_sales_global (global)")

## Creación de Tabla Persistente en Catálogo
Creamos un esquema llamado `sales_analytics` y guardamos la tabla en formato **Delta**.  
Este formato soporta **ACID transactions, versioning y optimizaciones**.

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS sales_analytics")

df_sales.write.format("delta").mode("overwrite").saveAsTable("sales_analytics.sales_raw")

print("✅ Tabla persistente creada: sales_analytics.sales_raw")

display(spark.catalog.listTables("sales_analytics"))

### Agregaciones y Joins
1. Resumen de ventas por cliente  
2. Catálogo de productos únicos  
3. Join con los **Top 10 clientes** y pedidos con `QUANTITYORDERED` superior al promedio global

In [0]:
agg_cliente = df_sales.groupBy("CUSTOMERNAME") \
                     .agg(F.sum("SALES").alias("total_sales"), \
                          F.countDistinct("ORDERNUMBER").alias("order_count")) \
                     .orderBy(F.desc("total_sales"))

display(agg_cliente.limit(10))

productos = df_sales.select("PRODUCTCODE", "PRODUCTLINE", "MSRP", "PRICEEACH").dropDuplicates(["PRODUCTCODE"])

top10_customers = [r["CUSTOMERNAME"] for r in agg_cliente.limit(10).collect()]
avg_qty = df_sales.agg(F.avg("QUANTITYORDERED")).collect()[0][0]

df_top_orders = df_sales.filter((F.col("CUSTOMERNAME").isin(top10_customers)) & (F.col("QUANTITYORDERED") > avg_qty))

display(df_top_orders.limit(20))

## Funciones de Ventana (Window Functions)
Aplicamos **row_number, dense_rank, lag y lead** para obtener diferentes perspectivas:

- `row_number`: la venta más alta por cliente  
- `dense_rank`: ranking de productos por línea de producto  
- `lag/lead`: comparación de ventas con pedido anterior y siguiente  

In [0]:
# row_number
w_customer_sales = Window.partitionBy("CUSTOMERNAME").orderBy(F.desc("SALES"))
df_top_sale_per_customer = df_sales.withColumn("rn", F.row_number().over(w_customer_sales)) \
                                     .filter(F.col("rn") == 1)

display(df_top_sale_per_customer.limit(10))

# dense_rank
prod_agg = df_sales.groupBy("PRODUCTCODE", "PRODUCTLINE") \
                   .agg(F.sum("SALES").alias("total_sale_product"))
prod_agg = prod_agg.withColumn("dense_rank", F.dense_rank().over(Window.partitionBy("PRODUCTLINE").orderBy(F.desc("total_sale_product"))))

display(prod_agg.filter(F.col("dense_rank") <= 5))

# lag / lead
w_customer_date = Window.partitionBy("CUSTOMERNAME").orderBy("ORDERDATE")
df_with_lag_lead = df_sales.withColumn("prev_sales", F.lag("SALES").over(w_customer_date)) \
                          .withColumn("next_sales", F.lead("SALES").over(w_customer_date))

display(df_with_lag_lead.select("CUSTOMERNAME", "ORDERNUMBER", "ORDERDATE", "SALES", "prev_sales", "next_sales").limit(20))